Check the status of tube lines

http://docs.python-requests.org/en/master/

http://docs.python-requests.org/en/master/user/quickstart/#make-a-request Tutorial

https://github.com/timcnicholls/transport_api_demo/blob/master/harwell_wantage_bus.py

In [1]:
import requests
import json
import pprint

In [2]:
#url = 'https://api.tfl.gov.uk/line/mode/tube/status'
url = 'https://api.tfl.gov.uk/line/{}/status'.format('bakerloo')
r = requests.get(url)  # get the response object

In [3]:
def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

In [7]:
bakerloo_dict = r.json()[0]
print_json(bakerloo_dict)

{'$type': 'Tfl.Api.Presentation.Entities.Line, Tfl.Api.Presentation.Entities',
 'created': '2017-06-15T12:37:35.513Z',
 'crowding': {'$type': 'Tfl.Api.Presentation.Entities.Crowding, '
                       'Tfl.Api.Presentation.Entities'},
 'disruptions': [],
 'id': 'bakerloo',
 'lineStatuses': [{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, '
                            'Tfl.Api.Presentation.Entities',
                   'created': '0001-01-01T00:00:00',
                   'id': 0,
                   'statusSeverity': 10,
                   'statusSeverityDescription': 'Good Service',
                   'validityPeriods': []}],
 'modeName': 'tube',
 'modified': '2017-06-15T12:37:35.513Z',
 'name': 'Bakerloo',
 'routeSections': [],
 'serviceTypes': [{'$type': 'Tfl.Api.Presentation.Entities.LineServiceTypeInfo, '
                            'Tfl.Api.Presentation.Entities',
                   'name': 'Regular',
                   'uri': '/Line/Route?ids=Bakerloo&serviceTypes=Regu

In [8]:
bakerloo_dict['lineStatuses']

[{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, Tfl.Api.Presentation.Entities',
  'created': '0001-01-01T00:00:00',
  'id': 0,
  'statusSeverity': 10,
  'statusSeverityDescription': 'Good Service',
  'validityPeriods': []}]

In [10]:
bakerloo_dict['lineStatuses'][0]['statusSeverityDescription']

'Good Service'

In [11]:
lines = ['bakerloo',
 'central',
 'circle',
 'district',
 'hammersmith-city',
 'jubilee',
 'metropolitan',
 'northern',
 'piccadilly',
 'victoria',
 'waterloo-city']

Wrap in a class with a method for updating and another for returning line of interest. Follow https://github.com/timcnicholls/home-assistant/blob/transport-api/homeassistant/components/sensor/uk_transport.py

In [103]:
class LondonTubeSensor():    # Entity
    """
    Sensor that reads the status of a tube lines.
    """

    API_URL_BASE = "https://api.tfl.gov.uk/line/{}/status"
    ICON = 'mdi:subway'

    def __init__(self, line):
        """Initialize the sensor."""
        self._data = {}
        self._url = self.API_URL_BASE
        self._line = line
        self._state = 'Updating'

    @property
    def name(self):
        """Return the line name of the sensor."""
        return self._line

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    @property
    def unit_of_measurement(self):    # Do I need?
        """Return the unit this state is expressed in."""
        return ""

    def update(self):
        """Perform an API request."""

        try:
            response = requests.get(self._url.format(self._line.lower()))
            response.raise_for_status()
            self._data = response.json()[0]['lineStatuses']   # convert to json
            self._statuses = [status['statusSeverityDescription'] for status in self._data]            

            if 'Good Service' in self._statuses:   # if good status, this is the only status returned
                self._state = 'Good Service'
                self._description = 'Nothing to report'
            else:
                self._state = 'Disruptions' 
                self._description = [status['reason'] for status in self._data] # get the reasons
                
        except requests.RequestException as req_exc:
            print(
                'Invalid response from API: %s', req_exc
            )


In [104]:
bakerloo_sensor = LondonTubeSensor('Bakerloo')
circle_sensor = LondonTubeSensor('Circle')

In [105]:
circle_sensor.update()
print(circle_sensor.state)
print(circle_sensor._statuses)
print_json(circle_sensor._description)

Disruptions
['Part Suspended', 'Minor Delays', 'Part Suspended']
['Circle and Hammersmith & City lines - due to ongoing investigations at the '
 'site of the building fire near Latimer Road, the Circle and Hammersmith & '
 'City lines are suspended between Wood Lane and Edgware Road until further '
 'notice.\r\n'
 '<P>\r\n'
 'Trains will operate between Hammersmith and Wood Lane approximately every 15 '
 'minutes. Tickets are being accepted on local buses. Please check here for '
 'the latest information before you travel.\r\n'
 '</P>',
 'Circle Line: No service Wood Lane and  Edgware Road due to ongoing '
 'investigations at the site of the building fire near Latimer Road. MINOR '
 'DELAYS on the rest of the line. ',
 'Circle Line: No service Wood Lane and  Edgware Road due to ongoing '
 'investigations at the site of the building fire near Latimer Road. MINOR '
 'DELAYS on the rest of the line. ']


In [106]:
len(circle_sensor._description)

3

In [99]:
circle_sensor._statuses

['Part Suspended', 'Minor Delays', 'Part Suspended']

In [81]:
bakerloo_sensor.update()
print(bakerloo_sensor._statuses)

['Good Service']


In [67]:
len(circle_sensor._data[0])

11

In [72]:
len(circle_sensor._data[0]['lineStatuses'])

4

In [41]:
hammer_sensor = LondonTubeSensor('Hammersmith-City')
hammer_sensor.update()
hammer_sensor.state

'Part Suspended'

In [34]:
response = requests.get('https://api.tfl.gov.uk/line/bakerloo/status').json()
response

[{'$type': 'Tfl.Api.Presentation.Entities.Line, Tfl.Api.Presentation.Entities',
  'created': '2017-06-15T12:37:35.513Z',
  'crowding': {'$type': 'Tfl.Api.Presentation.Entities.Crowding, Tfl.Api.Presentation.Entities'},
  'disruptions': [],
  'id': 'bakerloo',
  'lineStatuses': [{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, Tfl.Api.Presentation.Entities',
    'created': '0001-01-01T00:00:00',
    'id': 0,
    'statusSeverity': 10,
    'statusSeverityDescription': 'Good Service',
    'validityPeriods': []}],
  'modeName': 'tube',
  'modified': '2017-06-15T12:37:35.513Z',
  'name': 'Bakerloo',
  'routeSections': [],
  'serviceTypes': [{'$type': 'Tfl.Api.Presentation.Entities.LineServiceTypeInfo, Tfl.Api.Presentation.Entities',
    'name': 'Regular',
    'uri': '/Line/Route?ids=Bakerloo&serviceTypes=Regular'}]}]

In [33]:
bakerloo_sensor.name

'Tube sensor'

Place sensors in list

In [21]:
desired_tubes = ['district', 'circle']
sensors = []

for tube in desired_tubes:
    sensors.append(LondonTubeSensor(tube))

In [22]:
desired_tubes

['district', 'circle']

In [23]:
sensors   # for add_devices

In [24]:
sensors[0].update()

In [25]:
sensors[0].state

'Good Service'